# Michael Navarro: Competency 2 - Project 1

In [ ]:
#----------------------------------------------------------------------------------------
#                                       Imports 
#----------------------------------------------------------------------------------------
import os
import pandas as pd
from pandas.plotting import scatter_matrix
from pandas.plotting import autocorrelation_plot
from pandas.plotting import lag_plot

#----------------------------------------------------------------------------------------
#                                   Global Constants
#----------------------------------------------------------------------------------------
HOME_DIR = os.getcwd()
DATASET_DIR = os.path.join(HOME_DIR, 'datasets')
GAME_SALES_FNAME = "game_sales_data.csv"

#----------------------------------------------------------------------------------------
#                                   Helper functions 
#----------------------------------------------------------------------------------------
def load_dataset(fname):
    ''' Returns a csv file as a numpy object'''
    file_path = os.path.join(DATASET_DIR, fname)
    return pd.read_csv(file_path, encoding='ANSI')
# end load_dataset()

In [ ]:
game_sales = pd.read_csv(GAME_SALES_FNAME, encoding='ANSI')

## Requirement #01: Take a Peek

In [ ]:
game_sales.head(10)

From the first 10 records we can see that we have some very solid data to play around with. Even though there are some text attributes, due to their uniformity, we can enumerate these attributes to get a numerical version that can be used for our data exploration process.

In [ ]:
# Factorize/Enumerate text cols for arithmetic
game_sales['Platform_Code'] = game_sales.Platform.factorize()[0]
game_sales['Publisher_Code'] = game_sales.Publisher.factorize()[0]
game_sales['Developer_Code'] = game_sales.Developer.factorize()[0]

## Requirement #02: Display Statistical Information

In [ ]:
print(f'Number of Rows: {len(game_sales)}')
print(f'Number of Cols: {len(game_sales.columns)}')

The last three attribute columns ini the following output are the enumerated versions of the pre-existing attributes. By enumerating them we are now able to see that there are a total of 36 platforms, a little over a thousand publishers and over three thousand developers. As someone who has been part of the gaming world, see the number of publishers so high was shocking!

In [ ]:
game_sales.describe()

## Requirement #03: Data Exploration

### The Problem with Nulls
During data exploration would be deciding what to do with Null/NaN values and their respective rows. With the data provided by the info() method, we can see that some columns like "User_Score" and "Critic_Score" have a lorge number of missing values. In this situation, it would not be a good idea to fill those missing values with a 0 as it will skew the data in a negative way. One option would be to remove the rows will Null/NaN values. Another option is to leave them in, but this may lead to data skewing down the line as well. The last option to exhaust would be to scrape another resource and fill the missing attributes before filtering them out. This would allow us to keep a larger percentage of our dataset.

In [ ]:
game_sales.info()

### Outliers

Outliers pose a large problem when it comes to datasets as they can muddy the results and relationships in the long run. The following scatter plot can clearly show us that there is one very glaring outlier in the dataset.

In [ ]:
game_sales.plot(kind = 'scatter', x = 'Year', y = 'Total_Shipped', figsize = (10, 8))
print('PLOT: 1')

The details of the outlier are shown below.

In [ ]:
# Get the record with the highest value for total_shipped
game_sales[game_sales.Total_Shipped == game_sales.Total_Shipped.max()]

### Relationships
After deciding what to do with the missing values in the dataset, the next step in data exploration would be to see what kind of relationships we can see.
The relationship of interest to me how critic scores relate to total shipped sales by year. That is, does a higher critic score average result in a higher number of total shipped sales? The following plots intend illuminate such a relationship.

In [ ]:
game_sales.boxplot(by = 'Year', column = 'Critic_Score', grid = False, figsize = (25, 15))
print('PLOT: 2')

Plot 3 shows something peculier. While it seemed that total game sales were on the rise, it took a sudden downward trajector after the apex in the year 2008. Plot 4, below, shows why this is the case. Another thing of note is that the year 2006, when the outlier occurred, is not the highest selling year, in fact it isn't even in the top 5 years.

In [ ]:
# Get the total number of sales per year
sales_by_year = game_sales.groupby(['Year']).Total_Shipped.sum().reset_index()
sales_by_year.plot(kind = 'bar', x = 'Year', y = 'Total_Shipped', figsize = (25, 15))
print('PLOT: 3')

Plot 4 shows that the reason for total sales declining after the year 2008 is because the dataset just doesn't have as much data for the proceeding years. At this point we are left to decide to keep the years with such low records.

In [ ]:
# Get the number of records per year
recs_per_year = game_sales['Year'].value_counts()
recs_per_year = recs_per_year.sort_index()
recs_per_year.plot(kind = 'bar', figsize = (25, 15))
print('PLOT: 4')

# Requirement 04: Data Augmentation

Data augmentation in machine learning can be incredibly useful becausee it can allow for a larger dataset that can result in a better trained model. This is even more important if we are trying to model something that has very limited real world data obtainable. There seems to be two main ways to augment a dataset with different ways of achieving the end goal. 

The first way is to just collect more real data and add it to an exisiting dataset. This can round out the dataset and, at the very least, allow for a larger dataset to be used in training. Adding more real data can also be used to fill in any missing values from the pre-existing dataset.

The second way is to create fake data. This has been done and continues to be done in statistics. By getting the standard deviation and means of a dataset, we can then add very "real" fake data to our dataset. If our dataset is composed of images, then we would then do a myriad of image manipulation methods to create "new" images to use as part of the dataset. Such methods include mirroring, altering the color balance, scaling etc.

For this dataset in question, both, adding more real data and fake data are viable solutions for augmenting the dataset.

The first way is to scrape another resource such as a website to add new real entries to the dataset. During this process we can have a focus on filling out the years that have less associated records than others. When adding to the dataset, we must also filter out all the duplicate records that will be found, but only after making sure that the original dataset record has all its attributes filled out either beforehand or with the new information scraped. To keep data consistent, we can take a left join approach and prioritize original data and only take in data that is associated with an attribute missing its value.

The second way of augmenting the dataset is a little more complicated. We would be creating fake records to add to the dataset. The complication occurs because we must make sure the fake data is as close to being real as possible. If we do not take precautions to ensure that the fake data is within real world boundaries it can dirty our dataset and make it unusable. One such way of creating fake data is to give find out the average of total shipped sales for games that had sequels. Using this we can get a number to use to create a sequel for games that don't have a sequel but sold enough copies to warrant one. To decide how many sales it had, and the score received we can use a random number generation algorithm that uses the average of each attribute and then deicides to add or subtract the standard deviation of that attribute up to a certain number of times.